In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

import pandas as pd

In [2]:
!pip install transformers

In [3]:
from transformers import pipeline

In [4]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [5]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [6]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [7]:
!pip install sentencepiece

In [8]:
from transformers import XLMRobertaTokenizer

class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('microsoft/Multilingual-MiniLM-L12-H384')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=15,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [9]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=256,
                          shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False)

In [10]:
from torch import nn
from transformers import BertModel

In [11]:
class RobertaClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained('microsoft/Multilingual-MiniLM-L12-H384')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(384, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [15]:
model = RobertaClassifier().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.linear.parameters(), lr=0.01)  # неполное обучение

In [16]:
for epoch_num in range(5):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

  0%|                                                                                          | 0/709 [00:00<?, ?it/s]

Epochs: 1 | Train Loss:  0.003         | Train Accuracy:  0.508         | Val Loss:  0.005         | Val Accuracy:  0.505


  0%|                                                                                          | 0/709 [00:00<?, ?it/s]

Epochs: 2 | Train Loss:  0.003         | Train Accuracy:  0.517         | Val Loss:  0.005         | Val Accuracy:  0.495


  0%|                                                                                          | 0/709 [00:00<?, ?it/s]

Epochs: 3 | Train Loss:  0.003         | Train Accuracy:  0.523         | Val Loss:  0.005         | Val Accuracy:  0.530


  0%|                                                                                          | 0/709 [00:00<?, ?it/s]

Epochs: 4 | Train Loss:  0.003         | Train Accuracy:  0.523         | Val Loss:  0.005         | Val Accuracy:  0.512


100%|████████████████████████████████████████████████████████████████████████████████| 709/709 [08:57<00:00,  1.32it/s]


Epochs: 5 | Train Loss:  0.003         | Train Accuracy:  0.527         | Val Loss:  0.005         | Val Accuracy:  0.578
